# Go Board
According to [this](http://moderndescartes.com/essays/implementing_go) link.

In [1]:
import itertools
from collections import namedtuple

N = 19
NN = N * N
WHITE, BLACK, EMPTY = 'O', 'X', '.'
EMPTY_BOARD = EMPTY * NN

def swap_colors(color):
    if color == BLACK:
        return WHITE
    elif color == WHITE:
        return BLACK
    else:
        return color

def flatten(c):
    return N * c[0] + c[1]

# Convention: coords that have been flattened have a "f" prefix
def unflatten(fc):
    return divmod(fc, N)

We'll also want an easy way to find all the neighbors of a coordinate. Since this is a frequent computation, we'll just cache the results as a list of length `N * N`.

In [2]:
def is_on_board(c):
    return c[0] % N == c[0] and c[1] % N == c[1]

def get_valid_neighbors(fc):
    x, y = unflatten(fc)
    possible_neighbors = ((x+1, y), (x-1, y), (x, y+1), (x, y-1))
    return [flatten(n) for n in possible_neighbors if is_on_board(n)]

# Neighbors are indexed by flat coordinates
NEIGHBORS = [get_valid_neighbors(fc) for fc in range(NN)]

Next, let's define the useful concept of *"reach"*, as given in the Tromp-Taylor rules. Reach is essentially the set of points neighboring a chain of stones.

Reach is a useful concept in two ways: we want to know if a chain of stones can reach an empty point (the capture rule), and we want to know if a chain of empty spaces reaches black, white, or both colors (for assigning ownership of territory at the end).

To deduce the reach of a stone, we use a flood-fill type algorithm to simultaneously discover the entire chain, as well as all of that chain's neighboring colors (the reach). We'll return both the chain and its reach, since both are useful.

In [3]:
def find_reached(board, fc):
    color = board[fc]
    chain = set([fc])
    reached = set()
    frontier = [fc]
    while frontier:
        current_fc = frontier.pop()
        chain.add(current_fc)
        for fn in NEIGHBORS[current_fc]:
            if board[fn] == color and not fn in chain:
                frontier.append(fn)
            elif board[fn] != color:
                reached.add(fn)
    return chain, reached

Now, we're ready to implement basic moves! We have to place our stone, and then handle any captures, prioritizing opponent captures over our own.

In [10]:
class IllegalMove(Exception): pass

def place_stone(color, board, fc):
    return board[:fc] + color + board[fc+1:]

def bulk_place_stones(color, board, stones):
    byteboard = bytearray(board, encoding='ascii') # create mutable version of board
    color = ord(color)
    for fstone in stones:
        byteboard[fstone] = color
    return byteboard.decode('ascii') # and cast back to string when done

def maybe_capture_stones(board, fc):
    chain, reached = find_reached(board, fc)
    if not any(board[fr] == EMPTY for fr in reached):
        board = bulk_place_stones(EMPTY, board, chain)
        return board, chain
    else:
        return board, []

But we're not done yet - recall that the ko rule prevents repeating board positions. Let's record a "ko" coordinate, which is either a flattened coordinate, or None (indicating that there is no ko to worry about). To bundle these two concepts together, we'll use python's handy namedtuple to define a Position.

Detecting kos is surprisingly easy, and is captured in a function `is_koish`. (See full code for details.). A ko occurs if `is_koish` is `True`, and exactly 1 stone has been captured. Now we just have to check that we aren't violating our ko constraint, and set the ko constraint in the returned position

In [5]:
def is_koish(board, fc):
    'Check if fc is surrounded on all sides by 1 color, and return that color'
    if board[fc] != EMPTY: return None
    neighbor_colors = {board[fn] for fn in NEIGHBORS[fc]}
    if len(neighbor_colors) == 1 and not EMPTY in neighbor_colors:
        return list(neighbor_colors)[0]
    else:
        return None

In [6]:
class Position(namedtuple('Position', ['board', 'ko'])):
    @staticmethod
    def initial_state():
        return Position(board=EMPTY_BOARD, ko=None)

    def get_board(self):
        return self.board

    def __str__(self):
        import textwrap
        return '\n'.join(textwrap.wrap(self.board, N))
    
    def play_move(self, fc, color):
        board, ko = self
        
        if fc == ko or board[fc] != EMPTY:
            print(self)
            raise IllegalMove

        possible_ko_color = is_koish(board, fc)
        new_board = place_stone(color, board, fc)

        opp_color = swap_colors(color)
        opp_stones = []
        my_stones = []
        for fn in NEIGHBORS[fc]:
            if new_board[fn] == color:
                my_stones.append(fn)
            elif new_board[fn] == opp_color:
                opp_stones.append(fn)

        opp_captured = 0
        for fs in opp_stones:
            new_board, captured = maybe_capture_stones(new_board, fs)
            opp_captured += len(captured)

        for fs in my_stones:
            new_board, captured = maybe_capture_stones(new_board, fs)

        if opp_captured == 1 and possible_ko_color == opp_color:
            new_ko = list(opp_captured)[0]
        else:
            new_ko = None

        return Position(new_board, new_ko)

    def score(self):
        board = self.board
        while EMPTY in board:
            fempty = board.index(EMPTY)
            empties, borders = find_reached(board, fempty)
            possible_border_color = board[list(borders)[0]]
            if all(board[fb] == possible_border_color for fb in borders):
                board = bulk_place_stones(possible_border_color, board, empties)
            else:
                # if an empty intersection reaches both white and black,
                # then it belongs to neither player. 
                board = bulk_place_stones('?', board, empties)
        return board.count(BLACK) - board.count(WHITE)

    def get_liberties(self):
        board = self.board
        liberties = bytearray(NN)
        for color in (WHITE, BLACK):
            while color in board:
                fc = board.index(color)
                stones, borders = find_reached(board, fc)
                num_libs = len([fb for fb in borders if board[fb] == EMPTY])
                for fs in stones:
                    liberties[fs] = num_libs
                board = bulk_place_stones('?', board, stones)
        return list(liberties)

Let's try to play

In [53]:
g = Position \
    .initial_state() \
    .play_move(104, BLACK) \
    .play_move(20, WHITE) \
    .play_move(0, BLACK) \
    .play_move(100, WHITE) \
    .play_move(2, BLACK) \
    .play_move(101, WHITE) \
    .play_move(19, BLACK) \
    .play_move(102, WHITE) \
    .play_move(21, BLACK) \
    .play_move(103, WHITE) \
    .play_move(38, BLACK) \
    .play_move(120, WHITE) \
    .play_move(39, BLACK) \
    .play_move(105, WHITE) \
    .play_move(200, BLACK) \
    .play_move(85, WHITE) \
    .play_move(202, BLACK) \
    .play_move(123, WHITE) \
    
print(g)
print("Score: {}".format(g.score()))

X.X................
XOX................
XX.................
...................
.........O.........
.....OOOO.O........
......O..O.........
...................
...................
...................
..........X.X......
...................
...................
...................
...................
...................
...................
...................
...................
Score: -2


In [56]:
g.get_board()

'X.X................XOX................XX.............................................O..............OOOO.O..............O..O............................................................................X.X..............................................................................................................................................................'